In [1]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
import math
import numpy as np
from collections import defaultdict
import collections
import json
from tqdm.auto import tqdm
import evaluate

k = 3
model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
batch_size = 16
max_length = 384
doc_stride = 128
languages = ['ar', 'ko', 'te']

dataset = load_dataset("coastalcph/tydi_xor_rc")

def add_example_id(example, idx):
    example["id"] = str(idx)
    return example

def format_answer(example):
    if isinstance(example["answer"], str) and example["answer"].strip().startswith("{"):
        try:
            ans_dict = json.loads(example["answer"])
            example["answer_start"] = ans_dict.get("answer_start", [-1])[0] if ans_dict.get("answer_start") else -1
            example["answer"] = ans_dict.get("text", [""])[0]
        except json.JSONDecodeError:
            example["answer_start"] = -1
            example["answer"] = ""
    elif isinstance(example["answer"], dict):
        example["answer_start"] = example["answer"].get("answer_start", [-1])[0]
        example["answer"] = example["answer"].get("text", [""])[0]
    return example

train_dataset = dataset["train"].filter(lambda example: example['lang'] in languages).map(add_example_id, with_indices=True).map(format_answer)
val_dataset = dataset["validation"].filter(lambda example: example['lang'] in languages).map(add_example_id, with_indices=True).map(format_answer)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        
        answer_start = examples["answer_start"][sample_index]
        answer_text = examples["answer"][sample_index]

        if answer_start == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answer_start
            end_char = start_char + len(answer_text)
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
    return tokenized_examples

tokenized_train_dataset = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []
    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
    return tokenized_examples

tokenized_val_dataset = val_dataset.map(prepare_validation_features, batched=True, remove_columns=val_dataset.column_names)

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"finetuned-qa",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()
    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        min_null_score = None
        valid_answers = []
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]

    return predictions

metric = evaluate.load("squad")

def compute_metrics(p):
    return metric.compute(predictions=p.predictions, references=p.label)

def post_processing_function(examples, features, predictions, stage="eval"):
    predictions = postprocess_qa_predictions(examples=examples, features=features, raw_predictions=predictions)
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    references = [{"id": ex["id"], "answers": {"text": [ex["answer"]], "answer_start": [ex["answer_start"]]}} for ex in examples]
    return metric.compute(predictions=formatted_predictions, references=references)

class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics

        if self.post_process_function is not None and self.compute_metrics is not None:
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = {**eval_preds}
            self.log(metrics)
        else:
            metrics = {}

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

for i in range(k):
    print(f"Training model {i+1}/{k}")
    trainer = QuestionAnsweringTrainer(
        model,
        args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        eval_examples=val_dataset,
        tokenizer=tokenizer,
        post_process_function=post_processing_function,
    )
    trainer.train()

    for lang in languages:
        print(f"Evaluating for language: {lang}")
        lang_val_dataset = val_dataset.filter(lambda example: example['lang'] == lang)
        lang_tokenized_val_dataset = lang_val_dataset.map(prepare_validation_features, batched=True, remove_columns=val_dataset.column_names)
        
        metrics = trainer.evaluate(eval_dataset=lang_tokenized_val_dataset, eval_examples=lang_val_dataset)
        print(f"Results for {lang}: {metrics}")

Using the latest cached version of the dataset since coastalcph/tydi_xor_rc couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\aarus\.cache\huggingface\datasets\coastalcph___tydi_xor_rc\default\0.0.0\6c31d1c2ec1e15e57d836141bb5adb8da7df9124 (last modified on Thu Sep 25 15:11:30 2025).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model 1/3


C:\Users\aarus\AppData\Local\Temp\ipykernel_31388\211724850.py:203: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `QuestionAnsweringTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
wandb: Currently logged in as: aarushsinha60 (chungimungi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,No log


Evaluating for language: ar


...

Results for ar: {}
Evaluating for language: ko
Results for ko: {}
Evaluating for language: te
Results for te: {}
Training model 3/3


C:\Users\aarus\AppData\Local\Temp\ipykernel_31388\211724850.py:203: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `QuestionAnsweringTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,No log


Evaluating for language: ar


Results for ar: {}
Evaluating for language: ko
Results for ko: {}
Evaluating for language: te
Results for te: {}


In [3]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

label_list = ["O", "ANS"]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

max_length = 384
doc_stride = 128


def create_token_labels(examples):
    # Build token-level labels for the context part only; question tokens get label -100
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offsets_mapping = tokenized.pop("offset_mapping")

    labels = []
    for i, offsets in enumerate(offsets_mapping):
        sequence_ids = tokenized.sequence_ids(i)
        sample_idx = sample_mapping[i]
        answer_start = examples["answer_start"][sample_idx]
        answer_text = examples["answer"][sample_idx]
        answer_end = -1 if answer_start == -1 else answer_start + len(answer_text)

        example_labels = []
        context_id = 1 if pad_on_right else 0
        for idx, offset in enumerate(offsets):
            if sequence_ids[idx] is None:
                example_labels.append(-100)
            elif sequence_ids[idx] != context_id:
                example_labels.append(-100)
            else:
                if answer_start == -1 or offset is None:
                    example_labels.append(label_to_id["O"])  # unanswerable → no tokens
                else:
                    start, end = offset
                    if start >= answer_end or end <= answer_start:
                        example_labels.append(label_to_id["O"])  # outside answer span
                    else:
                        example_labels.append(label_to_id["ANS"])  # overlaps answer span
        labels.append(example_labels)

    tokenized["labels"] = labels
    return tokenized


def compute_token_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    # Only evaluate on context tokens (labels != -100)
    true_labels = []
    pred_labels = []
    for p, l in zip(preds, labels):
        for pi, li in zip(p, l):
            if li != -100:
                true_labels.append(li)
                pred_labels.append(pi)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, labels=[label_to_id["ANS"]], average="binary")
    acc = accuracy_score(true_labels, pred_labels)
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": acc}


results_by_lang = {}
for lang in languages:
    print(f"Training token-classifier for language: {lang}")
    lang_train = train_dataset.filter(lambda ex: ex["lang"] == lang)
    lang_val = val_dataset.filter(lambda ex: ex["lang"] == lang)

    tokenized_lang_train = lang_train.map(create_token_labels, batched=True, remove_columns=lang_train.column_names)
    tokenized_lang_val = lang_val.map(create_token_labels, batched=True, remove_columns=lang_val.column_names)

    model_tc = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label=id_to_label, label2id=label_to_id)
    args_tc = TrainingArguments(
        output_dir=f"seq-lab-{lang}",
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_steps=50,
        save_strategy="no",
        report_to=[],
    )

    data_collator = DataCollatorForTokenClassification(tokenizer)

    trainer_tc = Trainer(
        model=model_tc,
        args=args_tc,
        train_dataset=tokenized_lang_train,
        eval_dataset=tokenized_lang_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_token_metrics,
    )

    trainer_tc.train()
    metrics = trainer_tc.evaluate()
    results_by_lang[lang] = metrics
    print(f"Results for {lang}: {metrics}")

results_by_lang


Training token-classifier for language: ar


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aarus\AppData\Local\Temp\ipykernel_31388\3895613878.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_tc = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.131100,0.101309,0.598485,0.045559,0.084673,0.972713


Results for ar: {'eval_loss': 0.10130859166383743, 'eval_precision': 0.5984848484848485, 'eval_recall': 0.04555940023068051, 'eval_f1': 0.08467309753483387, 'eval_accuracy': 0.972712603645775, 'eval_runtime': 2.9675, 'eval_samples_per_second': 146.926, 'eval_steps_per_second': 18.534, 'epoch': 1.0}
Training token-classifier for language: ko


Filter:   0%|          | 0/6335 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aarus\AppData\Local\Temp\ipykernel_31388\3895613878.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_tc = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.118100,0.078299,0.514286,0.015831,0.030717,0.980213


Results for te: {'eval_loss': 0.07829934358596802, 'eval_precision': 0.5142857142857142, 'eval_recall': 0.0158311345646438, 'eval_f1': 0.030716723549488054, 'eval_accuracy': 0.9802125065319631, 'eval_runtime': 2.5508, 'eval_samples_per_second': 152.108, 'eval_steps_per_second': 19.209, 'epoch': 1.0}


{'ar': {'eval_loss': 0.10130859166383743,
  'eval_precision': 0.5984848484848485,
  'eval_recall': 0.04555940023068051,
  'eval_f1': 0.08467309753483387,
  'eval_accuracy': 0.972712603645775,
  'eval_runtime': 2.9675,
  'eval_samples_per_second': 146.926,
  'eval_steps_per_second': 18.534,
  'epoch': 1.0},
 'ko': {'eval_loss': 0.11313216388225555,
  'eval_precision': 0.6306620209059234,
  'eval_recall': 0.11145320197044335,
  'eval_f1': 0.18942961800104657,
  'eval_accuracy': 0.9672723431227551,
  'eval_runtime': 2.3927,
  'eval_samples_per_second': 151.295,
  'eval_steps_per_second': 19.225,
  'epoch': 1.0},
 'te': {'eval_loss': 0.07829934358596802,
  'eval_precision': 0.5142857142857142,
  'eval_recall': 0.0158311345646438,
  'eval_f1': 0.030716723549488054,
  'eval_accuracy': 0.9802125065319631,
  'eval_runtime': 2.5508,
  'eval_samples_per_second': 152.108,
  'eval_steps_per_second': 19.209,
  'epoch': 1.0}}